In [1]:
    import mo_descriptor as md
import nn_frame as nn
import numpy as np
import subprocess
subprocess.run('export TF_INTRA_OP_PARALLELISM_THREADS=12', shell=True)

2022-09-05 23:44:10.832377: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-05 23:44:10.832506: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CompletedProcess(args='export TF_INTRA_OP_PARALLELISM_THREADS=12', returncode=0)

In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shif  t),) + lumo_pair.shape)

# homo_ = np.zeros(homo.shape)

# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_[:,0] = np.add(homo[:,0],0)
#         homo_[:,1] = np.add(homo[:,1],i)
#         homo_[:,2] = np.add(homo[:,2],j)
#         homo_[:,3] = np.add(homo[:,3],0)
        
#         homo_pair_ = md.MO_pair_descriptor(homo, homo_).make()
#         homo_pairs[idx] = homo_pair_

        
# def dir_mat(mat):
#     mat_shape = mat.shape
#     mat_ = mat.flatten()
#     for ii, i in enumerate(mat_):
#         if i > 1e-6:
#             mat_[ii] = 1
#         elif (i < 1e-6) and (i > -1e-6):
#             mat_[ii] = -1
#         elif i < -1e-6:
#             mat_[ii] = -1
#     return mat_.reshape(mat_shape)

# direct = dir_mat(homo_pair)

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i*direct[1])
#         homo_pairs[idx][2] = np.add(homo_pair[2],j*direct[2])
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('data/cdft-V1V2.dat')
c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2

In [4]:
# x1 = homo_pairs[:,0,:,:]
# x2 = homo_pairs[:,1,:,:]
# x3 = homo_pairs[:,2,:,:]
# x4 = homo_pairs[:,3,:,:]
# x = np.einsum('aij,aij,aij,aij->aij', x1, x2, x3, x4)

train_homo_pairs = homo_pairs
train_lumo_pairs = lumo_pairs[0:1200]

train_c_homo = -np.log(c_homo)
train_c_lumo = c_lumo[0:1200]

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [5]:
setting = {'activation':'tanh', 'nn_shape':(256,256,256), 'batch_size':1681, 'training_steps':50000000,\
'learning_rate': 0.000001, 'decay_rate':0.95, 'decay_per_steps':1000, 'save_step':1000, 'drop_rate':0, 'save_path':'./save',\
'seed':None, 'debug_traj': True}
NN = nn.NN(setting_dict=setting)
NN.train(train_homo_pairs,train_c_homo)

2022-09-05 23:44:12.587646: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-05 23:44:12.587773: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Deng-PC): /proc/driver/nvidia/version does not exist
2022-09-05 23:44:12.588366: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


loss:  8.67877865
training step:     0
loss:  7.46247721
training step:  1000
loss:  7.29858589
training step:  2000
loss:  7.15421534
training step:  3000
loss:  7.01879406
training step:  4000
loss:  6.88273621
training step:  5000
loss:  6.74740744
training step:  6000
loss:  6.61134672
training step:  7000
loss:  6.47506714
training step:  8000
loss:  6.33915424
training step:  9000
loss:  6.20345402
training step: 10000
loss:  6.06759214
training step: 11000
loss:  5.93141174
training step: 12000
loss:  5.79484034
training step: 13000
loss:  5.65792036
training step: 14000
loss:  5.52061749
training step: 15000
loss:  5.38291025
training step: 16000
loss:  5.24490881
training step: 17000
loss:  5.10673714
training step: 18000
loss:  4.96844912
training step: 19000
loss:  4.8301425
training step: 20000
loss:  4.69194508
training step: 21000
loss:  4.55392694
training step: 22000
loss:  4.41615438
training step: 23000
loss:  4.2786355
training step: 24000
loss:  4.1414423
training s

2022-09-06 06:31:32.690112: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled


loss:  1.49749019e-06
training step: 1148000
loss:  1.49140283e-06
training step: 1149000
loss:  1.49414984e-06
training step: 1150000
loss:  1.48980268e-06
training step: 1151000
loss:  1.48669653e-06
training step: 1152000
loss:  1.48069307e-06
training step: 1153000
loss:  1.48790627e-06
training step: 1154000
loss:  1.4820057e-06
training step: 1155000
loss:  1.48272034e-06
training step: 1156000
loss:  1.47807373e-06
training step: 1157000
loss:  1.52275368e-06
training step: 1158000
loss:  1.47668243e-06
training step: 1159000
loss:  1.47513117e-06
training step: 1160000
loss:  1.4691002e-06
training step: 1161000
loss:  1.46896707e-06
training step: 1162000
loss:  1.47975811e-06
training step: 1163000


2022-09-06 06:37:13.182434: W tensorflow/core/data/root_dataset.cc:163] Optimization loop failed: CANCELLED: Operation was cancelled


loss:  1.46573234e-06
training step: 1164000
loss:  1.46431444e-06
training step: 1165000
loss:  1.46230786e-06
training step: 1166000
loss:  1.46114235e-06
training step: 1167000
loss:  2.08759175e-05
training step: 1168000
loss:  1.46078435e-06
training step: 1169000
loss:  1.45729541e-06
training step: 1170000
loss:  1.4582663e-06
training step: 1171000
loss:  1.45114973e-06
training step: 1172000
loss:  1.45417152e-06
training step: 1173000
loss:  2.40811587e-05
training step: 1174000
loss:  1.44978503e-06
training step: 1175000
loss:  1.4466741e-06
training step: 1176000
loss:  1.4432776e-06
training step: 1177000
loss:  1.44636908e-06
training step: 1178000
loss:  1.44054275e-06
training step: 1179000
loss:  1.44211992e-06
training step: 1180000
loss:  1.43797877e-06
training step: 1181000
loss:  1.43985858e-06
training step: 1182000
loss:  1.4357845e-06
training step: 1183000
loss:  1.43509681e-06
training step: 1184000
loss:  1.44542582e-06
training step: 1185000
loss:  1.43114

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

print(np.mean((NN.model(train_homo_pairs, training=False)-c_homo)/c_homo))
x = np.linspace(0, 4, 41)
y = np.linspace(0, 4, 41)
X, Y = np.meshgrid(x, y)
Z = NN.model(train_homo_pairs, training=False).numpy().reshape((41,41))
Z1 = c_homo.reshape((41,41))

fix, ax = plt.subplots()
ax.contourf(x,y, np.exp(-Z))
ax.set_title('pred')

In [ ]:
NN.model.summary()

In [ ]:
print(homo[:,2])